# 4.2 - Office finder mongo version

#### Encontramos tu oficina

In [1]:
# librerias

from pymongo import MongoClient

import pandas as pd

import folium

import re

In [2]:
# conexion a mongo

cliente=MongoClient()

db=cliente.companies

In [7]:
def get_data():   # realizar query y traer datos
    
    query=db.companies.find({'$and': [ {'$or': [{'category_code': 'games_video'},
                                                {'category_code': 'software'},
                                                {'category_code': 'web'},
                                                {'category_code': 'mobile'},
                                                {'category_code': 'photo_video'},
                                                {'category_code': 'ecomerce'},
                                                {'category_code': 'search'},
                                                {'category_code': 'network_hosting'}]},
                                      
                                      {'founded_year': {'$gte': 1999}},
                                      {'offices': {'$ne': []}},
                                      {'offices.longitude': {'$ne': []}},
                                      {'offices.latitude': {'$ne': []}},
                                      {'offices.longitude': {'$ne': None}},
                                      {'offices.latitude': {'$ne': None}},
                                      {'deadpooled_year': None},
                                      {'number_of_employees': {'$ne': None}},
                                      {'total_money_raised': {'$ne': None}}]},

                            {'name': 1, '_id':0, 'offices.latitude': 1, 'offices.longitude': 1,
                             'founded_year': 1, 'category_code':1, 'number_of_employees':1,
                             'total_money_raised':1
                            })
    
    
    df=pd.json_normalize(data=query, record_path='offices',
                         meta=['category_code', 'name', 'number_of_employees',
                               'founded_year', 'total_money_raised'])
    
    
    # esto inserta una columna vacia en la posicion 2
    df.insert(2, 'geo_loc', pd.Series(list(range(len(df)))))
    
    # aqui relleno la columna con una lista de longitud y latitud
    df['geo_loc']=df.apply(lambda x: [x['longitude'], x['latitude']], axis=1)
    
    return df

In [8]:
get_data().head(10)

,latitude,longitude,geo_loc,category_code,name,number_of_employees,founded_year,total_money_raised
0,34.090368,-118.393064,"[-118.393064, 34.090368]",web,Geni,18,2006,$16.5M
1,40.746497,-74.009447,"[-74.0094471, 40.7464969]",games_video,Joost,0,2006,$45M
2,37.387845,-122.055197,"[-122.055197, 37.387845]",web,Plaxo,50,2002,$28.3M
3,37.778613,-122.395289,"[-122.395289, 37.778613]",search,Powerset,60,2006,$22.5M
4,34.017606,-118.487267,"[-118.487267, 34.017606]",web,Mahalo,40,2007,$21M
5,37.788482,-122.409173,"[-122.409173, 37.788482]",games_video,Kyte,40,2006,$23.4M
6,37.480999,-122.173887,"[-122.173887, 37.480999]",mobile,Jingle Networks,35,2005,$88.7M
7,40.726040,-73.995722,"[-73.995722, 40.72604]",web,Meetup,75,2002,$18.3M
8,37.390056,-122.067692,"[-122.067692, 37.390056]",photo_video,SmugMug,62,2002,$0
9,37.423390,-122.089951,"[-122.0899512, 37.4233896]",mobile,Jajah,110,2005,$33M


In [16]:
def exchange(col):
    
    res=[]
    
    number=1
    
    symbol=1
    
    for e in col:
        
        if e[-1]=='k': number=1e3
        elif e[-1]=='M': number=1e6
        elif e[-1]=='B': number=1e9
            
        if e[0]=='£': symbol=1.3
        elif e[0]=='€' : symbol=1.12
        
        n_col=re.findall('[0-9.]+', e)[0]

        
        res.append(int(float(n_col) * number * symbol))
        
    return res

In [17]:
exchange(['$28.3M'])

[28300000]

In [18]:
def classifying(date, employees):
    
    company_class=[]
    
    for i in range(len(date)):
        
        if date[i]>=2008 and employees[i]<100:
            company_class.append('StartUp')
        else:
            company_class.append('BigCorp')
            
    
    return company_class

In [19]:
# llmada a los datos

df=get_data()

print('Numero total de compañias a considerar: ', len(df))

Numero total de compañias a considerar:  2670


In [20]:
df.head()

,latitude,longitude,geo_loc,category_code,name,number_of_employees,founded_year,total_money_raised
0,34.090368,-118.393064,"[-118.393064, 34.090368]",web,Geni,18,2006,$16.5M
1,40.746497,-74.009447,"[-74.0094471, 40.7464969]",games_video,Joost,0,2006,$45M
2,37.387845,-122.055197,"[-122.055197, 37.387845]",web,Plaxo,50,2002,$28.3M
3,37.778613,-122.395289,"[-122.395289, 37.778613]",search,Powerset,60,2006,$22.5M
4,34.017606,-118.487267,"[-118.487267, 34.017606]",web,Mahalo,40,2007,$21M


In [21]:
df['n_total_money']=exchange(df.total_money_raised)

df['company_class']=classifying(df.founded_year, 
                                df.number_of_employees)

df.head()

,latitude,longitude,geo_loc,category_code,name,number_of_employees,founded_year,total_money_raised,n_total_money,company_class
0,34.090368,-118.393064,"[-118.393064, 34.090368]",web,Geni,18,2006,$16.5M,16500000,BigCorp
1,40.746497,-74.009447,"[-74.0094471, 40.7464969]",games_video,Joost,0,2006,$45M,45000000,BigCorp
2,37.387845,-122.055197,"[-122.055197, 37.387845]",web,Plaxo,50,2002,$28.3M,28300000,BigCorp
3,37.778613,-122.395289,"[-122.395289, 37.778613]",search,Powerset,60,2006,$22.5M,22500000,BigCorp
4,34.017606,-118.487267,"[-118.487267, 34.017606]",web,Mahalo,40,2007,$21M,21000000,BigCorp


In [ ]:
# insertar en mongo

#df.to_json('geoloc.json', orient='records', lines=True)

In [22]:
db.geoloc.insert_many(df.to_dict('records'))

In [23]:
db.geoloc.create_index([('geo_loc', '2dsphere')])

'geo_loc_2dsphere'

In [24]:
# geolocalizacion

def geoquery(df):
    
    list_money_mean=[]
    
    sb_ratio=[]
    
    n_companies=[]
    
    for i in range(len(df)):
        
        q=list(db.geoloc.find({'geo_loc': {'$near': {'$geometry': {
                                     
                                'type': 'Point',
                                'coordinates': df.geo_loc[i]},
                                
                                '$maxDistance': 1000}}}))
        
        
        long_list=len(q)
        
        # media de dinero
        money_mean=sum([q[j]['n_total_money'] for j in range(long_list)]) / long_list
        
        list_money_mean.append(money_mean)
        
        # startup-bigcorp
        
        big=sum(q[k]['company_class'].count('BigCorp') 
                if q[k]['company_class'].count('BigCorp')!=0 
                else 1 for k in range(long_list))
        
        start=sum(q[k]['company_class'].count('StartUp')  for k in range(long_list))
        
        # si sb_ratio=1 entonces big=start, si sb_ratio<1 hay más big, si sb_ratio> hay mas start
        sb_ratio.append(start/big)
        
        
        # nº compañias
        n_companies.append(long_list)
        
        
    df['around_companies']=n_companies
    
    df['sb_ratio']=sb_ratio
    
    df['money_density']=list_money_mean
    
    return df

In [26]:
df=geoquery(df)

df.head()

,latitude,longitude,geo_loc,category_code,name,number_of_employees,founded_year,total_money_raised,n_total_money,company_class,around_companies,sb_ratio,money_density
0,34.090368,-118.393064,"[-118.393064, 34.090368]",web,Geni,18,2006,$16.5M,16500000,BigCorp,1,0.000000,1.650000e+07
1,40.746497,-74.009447,"[-74.0094471, 40.7464969]",games_video,Joost,0,2006,$45M,45000000,BigCorp,8,0.375000,7.325250e+06
2,37.387845,-122.055197,"[-122.055197, 37.387845]",web,Plaxo,50,2002,$28.3M,28300000,BigCorp,2,0.000000,2.715000e+07
3,37.778613,-122.395289,"[-122.395289, 37.778613]",search,Powerset,60,2006,$22.5M,22500000,BigCorp,27,0.222222,1.105724e+07
4,34.017606,-118.487267,"[-118.487267, 34.017606]",web,Mahalo,40,2007,$21M,21000000,BigCorp,3,0.333333,8.733333e+06


In [41]:
def geomap(df, zoom_start=1.5):

    df.index=range(len(df)) 

    mapa=folium.Map(location=[0, 0],
                    tiles='openstreetmap', 
                    zoom_start=zoom_start)
    
    
    for i in range(1, len(df)):

        folium.CircleMarker([float(df.latitude[i]), float(df.longitude[i])], 
                            popup=str(df.name[i]), 
                            radius=1, 
                            icon=folium.Icon()).add_to(mapa)

    #mapa.save('images/geomap.html')
    
    return mapa

In [42]:
print('oficinas')

mapa=geomap(df.loc[:1500])

mapa

oficinas


In [43]:
print ('Compañias alrededor>=10, sb_ratio>0.45 y densidad monetaria>1.000.000$')


data=df[(df['around_companies']>=10) & (df['sb_ratio']>0.45) & (df['money_density']>1e6)]

mapa2=geomap(data)

mapa2

Compañias alrededor>=10, sb_ratio>0.45 y densidad monetaria>1.000.000$


In [46]:
print ('Compañias alrededor>=50, sb_ratio>0.25 y densidad monetaria>10.000.000$')


data=df[(df['around_companies']>=50) & (df['sb_ratio']>0.25) & (df['money_density']>1e7)]

mapa3=geomap(data)

mapa3

Compañias alrededor>=50, sb_ratio>0.25 y densidad monetaria>10.000.000$


In [52]:
print ('Compañias alrededor>=20, sb_ratio<1. y densidad monetaria>1.000.000$')


data=df[(df['around_companies']>20) & (df['sb_ratio']<1.) & (df['money_density']>1e6)]

mapa4=geomap(data)

mapa4

Compañias alrededor>=20, sb_ratio<1. y densidad monetaria>1.000.000$
